CHANGELOG

v0.0 11/9/2023
- initialize a dataframe for all the metrics
For each K:
    - load train npz and unravel true and pred
    - use true and pred to calculate all the metrics
    - save the metrics into central dataframe
    - do the same for valid

In [1]:
from jarvis.utils.general import gpus
gpus.autoselect()

import numpy as np, pandas as pd, tensorflow as tf, os, time
from pathlib import Path
from IPython.display import clear_output, HTML, Javascript, display
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score, average_precision_score

import sys  
sys.path.append('/home/jjlou/Jerry/jerry_packages')
from jerry_utils import restart_kernel, load_dataset, save_dataset
import jerry_losses, jerry_metrics 

[ 2023-11-25 06:22:45 ] CUDA_VISIBLE_DEVICES automatically set to: 2           


In [2]:
name = 'Unfrozen_EfficientNetV2L_v0.0'
root = '/home/jjlou/Jerry/wsi-arterio/arteriosclerotic_vessel_detection_and_fine_segmentation/Arteriolosclerosis_classification/data'

In [3]:
Valid0 = ['D-436','D-870', 'D-343']
Valid1 = ['D-297', 'D-916', 'UCI-15-12']
Valid2 = ['D-322', 'D-562']

Valid = [Valid0, Valid1, Valid2]

In [4]:
df_model = pd.DataFrame(index=range(len(Valid)))

df_model['Train confusion matrix'] = ''
df_model['Train false pos rate'] = ''
df_model['Train false neg rate'] = ''          
df_model['Train specificity'] = ''
df_model['Train recall'] = ''
df_model['Train precision'] = ''
df_model['Train accuracy'] = ''
df_model['Train F1'] = ''
df_model['Train ROC AUC'] = ''
df_model['Train PR AUC'] = ''

df_model['Valid confusion matrix'] = ''
df_model['Valid false pos rate'] = ''
df_model['Valid false neg rate'] = ''          
df_model['Valid specificity'] = ''
df_model['Valid recall'] = ''
df_model['Valid precision'] = ''
df_model['Valid accuracy'] = ''
df_model['Valid F1'] = ''
df_model['Valid ROC AUC'] = ''
df_model['Valid PR AUC'] = ''

In [5]:
for v in Valid:
    v_path = f"{root}/K{Valid.index(v)}"
    model_path = f'{v_path}/models/{name}_Color.BasicMorph.Aug_K{Valid.index(v)}.hdf5'
    load_path = f'{v_path}/models/{name}_Color.BasicMorph.Aug_K{Valid.index(v)}'
    
    i = Valid.index(v)
    
    train = np.load(f'{load_path}_train.npz')
    pred = train['pred']
    true = train['true']
    
    #train
    tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
    df_model['Train confusion matrix'][i] = confusion_matrix(true, pred)
    df_model['Train false pos rate'][i] = fp / (fp + tn)
    df_model['Train false neg rate'][i] = fn / (tp + fn)           
    df_model['Train specificity'][i] = tn / (tn + fp)
    df_model['Train recall'][i] = recall_score(true, pred)
    df_model['Train precision'][i] = precision_score(true, pred)
    df_model['Train accuracy'][i] = accuracy_score(true, pred)
    df_model['Train F1'][i] = f1_score(true, pred)
    df_model['Train ROC AUC'][i] = roc_auc_score(true, pred)
    df_model['Train PR AUC'][i] = average_precision_score(true, pred)
    
    valid = np.load(f'{load_path}_valid.npz')
    pred = valid['pred']
    true = valid['true']
    
    #valid
    tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
    df_model['Valid confusion matrix'][i] = confusion_matrix(true, pred)
    df_model['Valid false pos rate'][i] = fp / (fp + tn)
    df_model['Valid false neg rate'][i] = fn / (tp + fn)           
    df_model['Valid specificity'][i] = tn / (tn + fp)
    df_model['Valid recall'][i] = recall_score(true, pred)
    df_model['Valid precision'][i] = precision_score(true, pred)
    df_model['Valid accuracy'][i] = accuracy_score(true, pred)
    df_model['Valid F1'][i] = f1_score(true, pred)
    df_model['Valid ROC AUC'][i] = roc_auc_score(true, pred)
    df_model['Valid PR AUC'][i] = average_precision_score(true, pred)

In [6]:
# load model
model = tf.keras.models.load_model(model_path)

df_model['Trainable Params'] = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])

save_path = f'{root}/models_results'
df_model.to_csv(f'{save_path}/{name}_Color.BasicMorph.Aug.csv')